# Studio: Working with Databases in Python

For today's studio, we will be using the [TV Shows dataset](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney) from Kaggle. We have already downloaded the CSV for you.

You will use the watchlist you created to answer these questions:

1. **Which streaming services contain the shows you want to watch next?**
2. **Which streaming service is the best value based on the shows you want to watch?**

As you complete the different tasks in the studio, you may choose between using Pandas or SQL. 

**Remember**: we learned in our prep work that one is oftentimes more efficient at certain tasks than the other, so choose wisely!

## My Watchlist

If you would like, please use this space to make note of your watchlist by editing the text cell.

1. Downton Abbey
2. Bridgerton
3. Shadow and Bone
4. Great British Baking Show
5. Dickinson
6. The Mandalorian
7. Big Little Lies
8. Making the Cut
9. The Real Housewives of Beverly Hills
10. Father Brown

## Database Setup

Import the necessary libraries and create a dataframe from the provided CSV. 

Print the info out for the dataframe. 

After that, you may drop the column called `Unnamed: 0` and rename any columns with spaces or unusual characters in the names such as `"Disney+"`. 

Print out the info for the dataframe again to ensure your changes were made.

In [79]:
import sqlite3 as sl
import pandas as pd

shows_df = pd.read_csv("tv_shows.csv")

# shows_df.info()

shows_df = shows_df.drop(columns=['Unnamed: 0'])
shows_df = shows_df.rename(columns = {'Rotten Tomatoes': 'Rotten_Tomatoes', 'Prime Video': 'Prime', 'Disney+': 'Disney'})

shows_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5368 entries, 0 to 5367
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               5368 non-null   int64 
 1   Title            5368 non-null   object
 2   Year             5368 non-null   int64 
 3   Age              3241 non-null   object
 4   IMDb             4406 non-null   object
 5   Rotten_Tomatoes  5368 non-null   object
 6   Netflix          5368 non-null   int64 
 7   Hulu             5368 non-null   int64 
 8   Prime            5368 non-null   int64 
 9   Disney           5368 non-null   int64 
 10  Type             5368 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 461.4+ KB


With your dataframe at the ready, create a new database called `tv.db`. 

Add a new table to your database called `shows` using the data in the dataframe. 

In [80]:
con = sl.connect("tv.db")

shows_df.to_sql('shows', con)

5368

With your new table and database set up, print out the top 20 records in the `shows` table.

In [81]:
# pandas

print(shows_df.head(20))

# sql

with con:
    data = con.execute("SELECT * FROM shows WHERE ID <= 20")
    for row in data:
        print(row)

    ID                       Title  Year  Age    IMDb Rotten_Tomatoes  \
0    1                Breaking Bad  2008  18+  9.4/10         100/100   
1    2             Stranger Things  2016  16+  8.7/10          96/100   
2    3             Attack on Titan  2013  18+  9.0/10          95/100   
3    4            Better Call Saul  2015  18+  8.8/10          94/100   
4    5                        Dark  2017  16+  8.8/10          93/100   
5    6  Avatar: The Last Airbender  2005   7+  9.3/10          93/100   
6    7              Peaky Blinders  2013  18+  8.8/10          93/100   
7    8            The Walking Dead  2010  18+  8.2/10          93/100   
8    9                Black Mirror  2011  18+  8.8/10          92/100   
9   10          The Queen's Gambit  2020  18+  8.6/10          92/100   
10  11                  Mindhunter  2017  18+  8.6/10          90/100   
11  12                   Community  2009   7+  8.5/10          90/100   
12  13                      Narcos  2015  18+  8.8/

Now, create a new table called `watchlist` that has three fields:
1. id -> data type of `INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT`
2. title -> data type of `TEXT`
3. importance_rank -> data type of `INTEGER`

For the `importance_rank` field, rank each of your watchlist shows based on how much you want to see them next, `10` being the most important and `1` being the least important.

Then, insert each of the items from your watchlist into the new `watchlist` table, using the `executemany` method from our exercises.

Finally, select all the records from the `watchlist` table and print them out to the console.

In [82]:
with con:
    con.execute("""
        CREATE TABLE watchlist (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            importance_rank INTEGER
        );
    """)

    sql = 'INSERT INTO watchlist (id, title, importance_rank) values(?, ?, ?)'
    data = [
        (1, 'Downton Abbey', 8),
        (2, 'Bridgerton', 7),
        (3, 'Shadow and Bone', 5),
        (4, 'Great British Baking Show', 4),
        (5, 'Dickinson', 6),
        (6, 'The Mandalorian', 10),
        (7, 'Big Little Lies', 9),
        (8, 'Making the Cut', 2),
        (9, 'The Real Housewives of Beverly Hills', 3),
        (10, 'Father Brown', 1)
    ]

    con.executemany(sql, data)

    data = con.execute("SELECT * FROM watchlist")
    for row in data:
        print(row)

(1, 'Downton Abbey', 8)
(2, 'Bridgerton', 7)
(3, 'Shadow and Bone', 5)
(4, 'Great British Baking Show', 4)
(5, 'Dickinson', 6)
(6, 'The Mandalorian', 10)
(7, 'Big Little Lies', 9)
(8, 'Making the Cut', 2)
(9, 'The Real Housewives of Beverly Hills', 3)
(10, 'Father Brown', 1)


## Working with the Data

Using Pandas or SQL, find the answer to these 2 questions:
1. How many of the total shows (full csv list) are on each streaming service?
2. What percentage of these total shows is available on each streaming service?

**Hint**:

Use the pandas `query` method to filter the data, and then the Python `len` method to find it's length. [Relevant Link](https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/)

In [83]:
total_shows_count = len(shows_df)

netflix_count = len(shows_df.query('Netflix == 1'))
hulu_count = len(shows_df.query('Hulu == 1'))
prime_count = len(shows_df.query('Prime == 1'))
disney_count = len(shows_df.query('Disney == 1'))

# multiplying by 100 for presentational purposes
pct_netflix = (netflix_count / total_shows_count) * 100
pct_hulu = (hulu_count / total_shows_count) * 100
pct_prime = (prime_count / total_shows_count) * 100
pct_disney = (disney_count / total_shows_count) * 100

df_streamer_metrics = pd.DataFrame({
    'streaming service': ["Netflix", "Hulu", "Prime", "Disney"],
    'total shows on service': [netflix_count, hulu_count, prime_count, disney_count],
	'percentage of total shows available on service': [pct_netflix, pct_hulu, pct_prime, pct_disney]
})

df_streamer_metrics.head(4)



,streaming service,total shows on service,percentage of total shows available on service
0,Netflix,1971,36.717586
1,Hulu,1621,30.197466
2,Prime,1831,34.109538
3,Disney,351,6.538748



Now, join your watchlist data to the shows data using pandas or SQL. Verify that you joined the data correctly.

Using this related dataset, come up with analytic code that answers these questions:
1. The number of watchlist shows each streaming service has
2. The percentage of your overall watchlist each streaming service has


In [87]:
streaming_df = pd.read_sql("""
    SELECT watchlist.title, shows.Netflix, shows.Hulu, shows.Prime, shows.Disney 
    FROM watchlist JOIN shows ON shows.Title == watchlist.title;
    """, con)

streaming_df.head(10)


total_watchlist_count = len(pd.read_sql("SELECT * FROM watchlist", con))

netflix_count = len(streaming_df.query('Netflix == 1'))
hulu_count = len(streaming_df.query('Hulu == 1'))
prime_count = len(streaming_df.query('Prime == 1'))
disney_count = len(streaming_df.query('Disney == 1'))

# multiplying by 100 for presentational purposes
pct_netflix = (netflix_count / total_watchlist_count) * 100
pct_hulu = (hulu_count / total_watchlist_count) * 100
pct_prime = (prime_count / total_watchlist_count) * 100
pct_disney = (disney_count / total_watchlist_count) * 100

df_streamer_metrics = pd.DataFrame({
    'streaming service': ["Netflix", "Hulu", "Prime", "Disney"],
    'watchlist shows on service': [netflix_count, hulu_count, prime_count, disney_count],
	'percentage of watchlist shows available on service': [pct_netflix, pct_hulu, pct_prime, pct_disney]
})

df_streamer_metrics.head(4)



,streaming service,watchlist shows on service,percentage of watchlist shows available on service
0,Netflix,3,30.0
1,Hulu,1,10.0
2,Prime,2,20.0
3,Disney,1,10.0


## Results

Now that you have done your analysis, make note of the answers to the following questions by editing the text cell:

1. Was every show on your watchlist in the Kaggle dataset? Do you have any ideas as to why a show might not have been present?

- No, some of the shows are probably not licensed to be on any of the 4 platforms

2. Did you include a show or shows in your watchlist that is exclusive to one of the platforms? How might that have impacted your analysis?

- If more of our shows are exclusive to one platform, our analysis is going to show that platform as being a better value than the others

3. Which streaming service(s) offered the most shows on your watchlist? Which streaming service(s) offered the least?

- Netflix, Tied between Disney and Hulu

4. Based on the shows you want to watch and the results of your analysis, is there a streaming service you think would be a good fit for you?

- Netflix based on pure numbers

# Bonus Mission

We didn't end up using that `importance_rank` field, did we?

Well, that was intentional! 

Your bonus mission is come up with analysis that uses that field to determine, based on show importance rank and number of shows available, which platform you should subscribe to.

In [85]:
# Place code below

rank_df = pd.read_sql("""
    SELECT watchlist.title, watchlist.importance_rank, shows.Netflix, shows.Hulu, shows.Prime, shows.Disney 
    FROM watchlist JOIN shows ON shows.Title == watchlist.title;
    """, con)

rank_df.head(10)


netflix = rank_df.query("Netflix == 1")
netflix_sum_rank = netflix["importance_rank"].sum()

hulu = rank_df.query("Hulu == 1")
hulu_sum_rank = hulu["importance_rank"].sum()

prime = rank_df.query("Prime == 1")
prime_sum_rank = prime["importance_rank"].sum()

disney = rank_df.query("Disney == 1")
disney_sum_rank = disney["importance_rank"].sum()


most_valuable_service = pd.DataFrame({
    'Netlfix': [netflix_sum_rank],
	'Hulu': [hulu_sum_rank],
	'Prime': [prime_sum_rank],
	'Disney': [disney_sum_rank]
})

# The one with the highest number is the most valuable
most_valuable_service.head()

,Netlfix,Hulu,Prime,Disney
0,20,3,10,10
